PRIMER PASO: BARAJEAR DATOS

In [6]:
import pandas as pd
import random

# Leer el dataset original desde la carpeta data
df = pd.read_csv('data/initial_data.csv')

print(f"Dataset original cargado: {len(df)} noticias")
print(f"Distribución de sentimientos:")
print(df['Sentiment'].value_counts())
print("\n" + "="*60)

# Crear una copia para las noticias correctas (50%)
df_correctas = df.copy()
df_correctas['Etiqueta'] = 'correcta'

# Separar por sentimientos para mezclas
df_positive = df[df['Sentiment'] == 'positive']
df_negative = df[df['Sentiment'] == 'negative']

print(f"\nNoticias positivas disponibles: {len(df_positive)}")
print(f"Noticias negativas disponibles: {len(df_negative)}")

# Función para extraer fragmentos de una noticia
def extraer_fragmentos(noticia):
    """
    Extrae fragmentos de una noticia dividiéndola por comas o puntos
    """
    # Primero intentar dividir por comas
    fragmentos = [p.strip() for p in noticia.split(',') if p.strip()]
    
    # Si no hay suficientes fragmentos, dividir por puntos
    if len(fragmentos) <= 1:
        fragmentos = [p.strip() for p in noticia.split('.') if p.strip()]
    
    return fragmentos

# Función para crear mezclas de sentimientos OPUESTOS
def crear_mezclas_opuestas(df_positive, df_negative, num_mezclas):
    """
    Crea mezclas incorrectas combinando noticias positivas con negativas
    """
    noticias_mezcladas = []
    
    print(f"\nCreando {num_mezclas} mezclas de sentimientos opuestos...")
    
    for i in range(num_mezclas):
        if (i + 1) % 1000 == 0:
            print(f"  Procesadas {i + 1}/{num_mezclas}...")
        
        # Seleccionar una noticia positiva y una negativa
        idx_pos = random.randint(0, len(df_positive) - 1)
        idx_neg = random.randint(0, len(df_negative) - 1)
        
        noticia_pos = df_positive.iloc[idx_pos]['Sentence']
        noticia_neg = df_negative.iloc[idx_neg]['Sentence']
        
        # Extraer fragmentos
        frag_pos = extraer_fragmentos(noticia_pos)
        frag_neg = extraer_fragmentos(noticia_neg)
        
        # Seleccionar 1-2 fragmentos de cada una
        if len(frag_pos) > 0 and len(frag_neg) > 0:
            num_pos = min(random.randint(1, 2), len(frag_pos))
            num_neg = min(random.randint(1, 2), len(frag_neg))
            
            fragmentos_seleccionados = (
                random.sample(frag_pos, num_pos) + 
                random.sample(frag_neg, num_neg)
            )
            
            # Mezclar el orden
            random.shuffle(fragmentos_seleccionados)
            
            noticia_mezclada = ', '.join(fragmentos_seleccionados)
            if noticia_mezclada and noticia_mezclada[-1] not in '.!?':
                noticia_mezclada += '.'
            
            # Asignar sentimiento aleatorio entre los dos opuestos
            sentiment = random.choice(['positive', 'negative'])
            
            noticias_mezcladas.append({
                'Sentence': noticia_mezclada,
                'Sentiment': sentiment,
                'Etiqueta': 'incorrecta'
            })
    
    return pd.DataFrame(noticias_mezcladas)

# Crear el mismo número de noticias incorrectas que correctas
num_mezclas = len(df_correctas)
df_incorrectas = crear_mezclas_opuestas(df_positive, df_negative, num_mezclas)

# Combinar ambos datasets
df_final = pd.concat([df_correctas, df_incorrectas], ignore_index=True)

# Mezclar aleatoriamente el dataset final
df_final = df_final.sample(frac=1, random_state=42).reset_index(drop=True)

# Mostrar estadísticas
print(f"\n{'='*60}")
print(f"ESTADÍSTICAS DEL DATASET FINAL")
print(f"{'='*60}")
print(f"Total de filas: {len(df_final)}")
print(f"Noticias correctas: {len(df_final[df_final['Etiqueta'] == 'correcta'])} ({len(df_final[df_final['Etiqueta'] == 'correcta'])/len(df_final)*100:.1f}%)")
print(f"Noticias incorrectas: {len(df_final[df_final['Etiqueta'] == 'incorrecta'])} ({len(df_final[df_final['Etiqueta'] == 'incorrecta'])/len(df_final)*100:.1f}%)")

print(f"\nDistribución de sentimientos en noticias CORRECTAS:")
print(df_final[df_final['Etiqueta'] == 'correcta']['Sentiment'].value_counts())

print(f"\nDistribución de sentimientos en noticias INCORRECTAS:")
print(df_final[df_final['Etiqueta'] == 'incorrecta']['Sentiment'].value_counts())

# Mostrar ejemplos
print(f"\n{'='*60}")
print(f"EJEMPLOS")
print(f"{'='*60}")

print("\n[NOTICIA CORRECTA]")
ejemplo_correcta = df_final[df_final['Etiqueta'] == 'correcta'].iloc[0]['Sentence']
print(f"Sentimiento: {df_final[df_final['Etiqueta'] == 'correcta'].iloc[0]['Sentiment']}")
print(ejemplo_correcta[:300] + ("..." if len(ejemplo_correcta) > 300 else ""))

print("\n[NOTICIA INCORRECTA - Ejemplo 1]")
ejemplo_inc1 = df_final[df_final['Etiqueta'] == 'incorrecta'].iloc[0]['Sentence']
print(f"Sentimiento: {df_final[df_final['Etiqueta'] == 'incorrecta'].iloc[0]['Sentiment']}")
print(ejemplo_inc1[:300] + ("..." if len(ejemplo_inc1) > 300 else ""))

print("\n[NOTICIA INCORRECTA - Ejemplo 2]")
ejemplo_inc2 = df_final[df_final['Etiqueta'] == 'incorrecta'].iloc[50]['Sentence']
print(f"Sentimiento: {df_final[df_final['Etiqueta'] == 'incorrecta'].iloc[50]['Sentiment']}")
print(ejemplo_inc2[:300] + ("..." if len(ejemplo_inc2) > 300 else ""))

print("\n[NOTICIA INCORRECTA - Ejemplo 3]")
ejemplo_inc3 = df_final[df_final['Etiqueta'] == 'incorrecta'].iloc[100]['Sentence']
print(f"Sentimiento: {df_final[df_final['Etiqueta'] == 'incorrecta'].iloc[100]['Sentiment']}")
print(ejemplo_inc3[:300] + ("..." if len(ejemplo_inc3) > 300 else ""))

# Guardar el dataset final
output_path = 'data/dataset_mezclado_final.csv'
df_final.to_csv(output_path, index=False, encoding='utf-8')
print(f"\n{'='*60}")
print(f"✓ Dataset guardado como '{output_path}'")
print(f"{'='*60}")

Dataset original cargado: 5842 noticias
Distribución de sentimientos:
Sentiment
neutral     3130
positive    1852
negative     860
Name: count, dtype: int64


Noticias positivas disponibles: 1852
Noticias negativas disponibles: 860

Creando 5842 mezclas de sentimientos opuestos...
  Procesadas 1000/5842...
  Procesadas 2000/5842...
  Procesadas 3000/5842...
  Procesadas 4000/5842...
  Procesadas 5000/5842...

ESTADÍSTICAS DEL DATASET FINAL
Total de filas: 11684
Noticias correctas: 5842 (50.0%)
Noticias incorrectas: 5842 (50.0%)

Distribución de sentimientos en noticias CORRECTAS:
Sentiment
neutral     3130
positive    1852
negative     860
Name: count, dtype: int64

Distribución de sentimientos en noticias INCORRECTAS:
Sentiment
positive    2939
negative    2903
Name: count, dtype: int64

EJEMPLOS

[NOTICIA CORRECTA]
Sentimiento: neutral
The presentation material can be viewed on the company 's website in English after the conference .

[NOTICIA INCORRECTA - Ejemplo 1]
Sentimiento: neg

TRADUCCIÓN

In [2]:
# Instalar librerías necesarias
!pip install deep-translator pandas tqdm

import pandas as pd
from deep_translator import GoogleTranslator
import random
from tqdm import tqdm
import time
import os

# Leer el dataset original
df = pd.read_csv('data/dataset_mezclado_final.csv')

# Definir los idiomas a los que traducir (códigos ISO)
idiomas = ['en', 'fr', 'de', 'it', 'pt', 'ca', 'eu', 'gl']  # inglés, francés, alemán, italiano, portugués, catalán, euskera, gallego

# Diccionario para nombres completos de idiomas
nombres_idiomas = {
    'es': 'español',
    'en': 'inglés',
    'fr': 'francés',
    'de': 'alemán',
    'it': 'italiano',
    'pt': 'portugués',
    'ca': 'catalán',
    'eu': 'euskera',
    'gl': 'gallego'
}

# Crear una copia del dataframe
df_traducido = df.copy()

# Crear nueva columna para el idioma
df_traducido['Idioma'] = ''

# Función para traducir con manejo de errores
def traducir_texto(texto, idioma_destino):
    try:
        if idioma_destino == 'es':  # Si es español, no traducir
            return texto
        translator = GoogleTranslator(source='es', target=idioma_destino)
        traduccion = translator.translate(texto)
        time.sleep(0.5)  # Pausa para evitar límites de la API
        return traduccion
    except Exception as e:
        print(f"Error traduciendo a {idioma_destino}: {e}")
        return texto  # Si falla, devolver el texto original

# Traducir SOLO la primera columna (Sentence) de cada fila a un idioma aleatorio
print("Iniciando traducción del dataset...")
print(f"Total de filas a procesar: {len(df_traducido)}")

for idx in tqdm(range(len(df_traducido))):
    # Seleccionar un idioma aleatorio (incluyendo español para mantener algunas filas originales)
    idioma_elegido = random.choice(idiomas + ['es', 'es', 'es'])  # Mayor probabilidad de español
    
    # Traducir SOLO la columna 'Sentence' (primera columna)
    # Las columnas 'Sentiment' y 'Etiqueta' permanecen iguales
    texto_original = df_traducido.loc[idx, 'Sentence']
    df_traducido.loc[idx, 'Sentence'] = traducir_texto(texto_original, idioma_elegido)
    
    # Añadir el idioma a la nueva columna
    df_traducido.loc[idx, 'Idioma'] = nombres_idiomas[idioma_elegido]



# Guardar el nuevo dataset
output_path = 'data/dataset_multiidioma.csv'
df_traducido.to_csv(output_path, index=False, encoding='utf-8')

print(f"\n✓ Dataset traducido guardado en: {output_path}")
print(f"Total de filas: {len(df_traducido)}")
print(f"\nEstructura del dataset:")
print(df_traducido.head())
print(f"\nDistribución de idiomas:")
print(df_traducido['Idioma'].value_counts())

Defaulting to user installation because normal site-packages is not writeable
Iniciando traducción del dataset...
Total de filas a procesar: 11684


  0%|          | 8/11684 [00:11<4:37:29,  1.43s/it]


KeyboardInterrupt: 